`Biblioteki`

In [1]:
import pandas as pd
import os

import itertools as it
from itertools import repeat

# sktime
import sktime
from sktime.forecasting.arima import AutoARIMA
from sktime.forecasting.base import ForecastingHorizon
from sktime.forecasting.compose import make_reduction
from sktime.forecasting.compose import TransformedTargetForecaster, EnsembleForecaster
from sktime.forecasting.fbprophet import Prophet
from sktime.forecasting.model_selection import (ForecastingRandomizedSearchCV, ForecastingGridSearchCV, SingleWindowSplitter)
from sktime.forecasting.model_selection import temporal_train_test_split
from sktime.forecasting.model_selection._split import temporal_train_test_split
from sktime.forecasting.trend import PolynomialTrendForecaster, STLForecaster

# plotly
import plotly.express as px
import plotly.graph_objects as go

`Funkcje`

In [2]:
def wykres(y_train ,y_test = None, y_pred = None, y_pred_intervals = None):
    # Wyplotowanie wyników
    fig = px.line(liquidation_store_y.VALUE)
    fig = go.Figure()
    fig.add_trace(
        go.Scatter(
            x=y_train.index,
            y=y_train['VALUE'],
            mode='lines+markers',
            name='TRAIN',
            line=dict(color='rgba(52,52,215,0.75)')

        )
    )
    if y_test is not None:
        fig.add_trace(
            go.Scatter(
                x=y_test.index,
                y=y_test['VALUE'],
                mode='lines+markers',
                name='TEST',
                line=dict(color='rgba(52,215,52,0.75)')
            )
        )
    if y_pred is not None:
        fig.add_trace(
            go.Scatter(
                x=y_pred.index,
                y=y_pred['VALUE'],
                name='PRED',
                mode='lines+markers',
                line=dict(color='rgba(235,52,52,0.75)')
            )
        )
    
    if y_pred_intervals is not None:
        fig.add_trace(
            go.Scatter(
                x=list(it.chain.from_iterable([y_pred_intervals.index, y_pred_intervals.index[::-1]])), # x, then x reversed
                y=list(it.chain.from_iterable([y_pred_intervals['upper'], y_pred_intervals['lower'][::-1]])), # upper, then lower reversed
                fill='toself',
                fillcolor='rgba(68, 68, 68, 0.3)',
                line=dict(color='rgba(255,255,255,0)'),
                name='CONFINT'
            )
    )
    return fig    
    #fig.show()

def compute_metrics(y_true, y_pred):
    mape = mean_absolute_percentage_error(y_true, y_pred, symmetric=False)
    mspe = mean_squared_percentage_error(y_true, y_pred, symmetric=False)
    _rmse = MeanSquaredError(square_root=True)
    rmse = _rmse(y_true, y_pred)
    ramka = pd.DataFrame([[mape],[mspe],[rmse]], index= ['MAPE','MSPE','RMSE'])
    ramka.columns = ['wynik']
    return ramka


`Funkcje do konwertowania`

In [31]:
def convert_and_preprocess_daily_to_monthly(pred):
    try:
        selected_index = pred.groupby(pd.Grouper(freq='M')).count().query("VALUE > 25").index
    except:
        selected_index = pred.groupby(pd.Grouper(freq='M')).count().query("upper > 25").index
    return pred.groupby(pd.Grouper(freq='M')).sum().loc[selected_index,:]


def convert_weekly_to_monthly(pred):
    _pred = convert_and_preprocess_daily_to_monthly(convert_weekly_to_daily(pred))
    return _pred

def convert_weekly_to_daily(preds):
    _preds = preds.copy()
    _preds.index = _preds.index.shift(-6, freq='D')
    _preds = _preds.resample('D').ffill()
    _preds['TOTAL_SALES'] = _preds['TOTAL_SALES'] / 7
    _preds['upper'] = _preds['upper'] / 7
    _preds['lower'] = _preds['lower'] / 7
    _preds = pd.concat([_preds,pd.DataFrame(index = _preds.index[-6:].shift(6))]).ffill()
    return _preds

`Kalendarz`

In [ ]:
kalendarz = pd.read_excel('', sheet_name= '')
kalendarz.tail(5)

`Dotyczczasowa inflacja`

In [4]:
inflacja = pd.read_csv("https://stat.gov.pl/download/gfx/portalinformacyjny/pl/defaultstronaopisowa/4741/1/1/miesieczne_wskazniki_cen_towarow_i_uslug_konsumpcyjnych_od_1982_roku.csv", encoding = 'windows-1250', sep=';', decimal=',')

# Usunięcie zbędnych kolumn
inflacja = inflacja.iloc[:,:6]

# Stworzenie kolumny z przyjaznym formatem i nowej z opóźnieniem zmiennej
inflacja['Wskaźnik_inflacji'] = inflacja['Wartość'] - 100
inflacja['Wskaźnik_inflacji_lag'] = inflacja['Wskaźnik_inflacji'].shift(1)

# Obróbka ramki i kolumn
inflacja = inflacja[(inflacja['Sposób prezentacji'] == 'Analogiczny miesiąc poprzedniego roku = 100') & (inflacja['Rok']>2015)]
inflacja['Miesiąc'] = inflacja.Miesiąc.map("{:02}".format)
inflacja['CALMONTH'] = inflacja.Rok.astype(str) + inflacja['Miesiąc']
inflacja['CALMONTH'] = inflacja['CALMONTH'].astype(int)

inflacja.head(5)

,Nazwa zmiennej,Jednostka terytorialna,Sposób prezentacji,Rok,Miesiąc,Wartość,Wskaźnik_inflacji,Wskaźnik_inflacji_lag,CALMONTH
24,Wskaźnik cen towarów i usług konsumpcyjnych,Polska,Analogiczny miesiąc poprzedniego roku = 100,2022,01,109.4,9.4,0.1,202201
25,Wskaźnik cen towarów i usług konsumpcyjnych,Polska,Analogiczny miesiąc poprzedniego roku = 100,2022,02,108.5,8.5,9.4,202202
26,Wskaźnik cen towarów i usług konsumpcyjnych,Polska,Analogiczny miesiąc poprzedniego roku = 100,2022,03,111.0,11.0,8.5,202203
27,Wskaźnik cen towarów i usług konsumpcyjnych,Polska,Analogiczny miesiąc poprzedniego roku = 100,2022,04,112.4,12.4,11.0,202204
28,Wskaźnik cen towarów i usług konsumpcyjnych,Polska,Analogiczny miesiąc poprzedniego roku = 100,2022,05,113.9,13.9,12.4,202205


`Dane makroekonomiczne`

In [5]:
makroekonomiczne = pd.read_excel('', sheet_name= '').set_index('index').stack().reset_index().rename(columns={'level_1':'kwartał',0:'wartość'})
makroekonomiczne = makroekonomiczne[~makroekonomiczne.kwartał.isin([...])]

# Usunięcie niepotrzebnych spacji w wartościach
makroekonomiczne = makroekonomiczne.apply(lambda x: x.str.strip() if x.dtype == "object" else x)

# Zamiana tekstu na datę - kwartał
makroekonomiczne['kwartał'] =  pd.to_datetime(makroekonomiczne['kwartał']).dt.to_period('Q')

# Stworzenie ostatecznej tabelki 
makroekonomiczne = makroekonomiczne.pivot_table('wartość', 'kwartał','index').reset_index()

makroekonomiczne.tail(5)

C:\ProgramData\Anaconda3\lib\site-packages\openpyxl\worksheet\header_footer.py:48: UserWarning: Cannot parse header or footer so it will be ignored
  warn("""Cannot parse header or footer so it will be ignored""")


index,kwartał,"Ceny energii (%, r/r)","Ceny żywności (%, r/r)","Deflator wartości dodanej za granicą (%, r/r)","Eksport (%, r/r)","Import (%, r/r)","Indeks cen surowców energetycznych (USD; 2011=1,00)","Indeks cen surowców rolnych (EUR; 2011=1,00)","Inflacja bazowa (CPI po wyłączeniu cen żywności i energii, %, r/r)","Jednostkowe koszty pracy (%, r/r)",...,Saldo rachunku bieżącego (% PKB),"Spożycie gospodarstw domowych (%, r/r)","Spożycie publiczne (%, r/r)",Stopa aktywności zawodowej (%),Stopa bezrobocia (%),WIBOR 3M (%),"Wkład eksportu netto (pkt proc., r/r)","Wskaźnik cen konsumenta CPI (%, r/r)","Wydajność pracy (%, r/r)","Wynagrodzenia (%, r/r)"
23,2024Q4,6.8,3.4,2.3,4.1,2.0,1.74,1.22,5.0,2.9,...,-0.1,1.0,1.1,57.6,4.8,6.95,1.4,4.9,3.6,6.7
24,2025Q1,6.4,2.5,2.0,4.0,3.1,1.76,1.22,4.3,2.3,...,0.1,1.8,1.9,57.5,4.8,6.95,0.8,4.2,3.9,6.4
25,2025Q2,5.8,2.0,1.8,3.8,3.6,1.63,1.24,3.8,2.5,...,0.2,1.7,2.4,57.6,4.8,6.95,0.4,3.6,3.3,5.9
26,2025Q3,5.3,1.6,1.7,3.9,3.8,1.52,1.22,3.4,1.9,...,0.3,1.7,3.1,57.5,4.8,6.95,0.4,3.3,3.7,5.6
27,2025Q4,5.0,1.5,1.7,4.1,3.5,1.49,1.22,3.1,2.1,...,0.5,1.5,3.2,57.5,4.9,6.95,0.6,3.0,3.4,5.6


`Wczytanie danych sprzedażowych`

In [6]:
path = '...' 
paragony= pd.concat([pd.read_parquet(path+part) for part in os.listdir(path)])

# KOLUMNA Z MIESIĄCEM - czyszczenie kolumn
paragony['CALMONTH'] = paragony['CALDAY'].apply(lambda x: str(x)[:-2]).astype(int)
paragony['ARTICLE'] = pd.to_numeric(paragony.ARTICLE)
paragony['CALDAY'] = pd.to_datetime(paragony['CALDAY'])

# Grupowanie
paragony = paragony.groupby(['STORE','CALMONTH','CALDAY'])[['STORE','TOTAL_SALES', 'TOTAL_VOLUME_SOLD', 'CALDAY', 'CALMONTH']].agg({'TOTAL_SALES':'sum','TOTAL_VOLUME_SOLD': 'sum'}).reset_index()
paragony['STORE'] = paragony['STORE'].astype(int)

In [8]:
# Rozpoczęcie od kalendarza i dalsza obudowa
start_kal , stop_kal = paragony.CALDAY.sort_values()[0], paragony.CALDAY.sort_values()[len(paragony)-1]

# Połączenie
par = []
for sklep in paragony.STORE.unique():
    dla_sklepu = paragony[paragony.STORE == sklep]
    tab = pd.merge(kalendarz[(kalendarz.data >= start_kal)&(kalendarz.data <= '')]
                   , dla_sklepu 
                   , left_on='data'
                   , right_on='CALDAY'
                   , how='left')
    tab['STORE'] = sklep
    tab['CALDAY'] = tab['data']
    tab['CALMONTH'] = (tab['CALDAY'].dt.year.astype(str) + tab['CALDAY'].dt.month.map("{:02}".format).astype(str)).astype(int)
    par.append(tab)
paragony = pd.concat(par)

# Nałożenie inflacji
paragony = paragony.merge(inflacja[['Wartość', 'Wskaźnik_inflacji', 'Wskaźnik_inflacji_lag', 'CALMONTH']], on = 'CALMONTH', how='left').rename(columns = {'Wartość':'Wartość inflacji'})

# Dodanie do paragononów kolumny z kwartałem
paragony['kwartał'] = pd.PeriodIndex(paragony.CALDAY, freq='Q')

# Nałożenie zmiennych makroekonomicznych
paragony = paragony.merge(makroekonomiczne, on='kwartał', how = 'left')

### Trenowanie modelu i wybór najlepszego

`Analiza wizualna`

In [ ]:
fig = px.line(paragony.set_index('CALDAY').groupby(pd.Grouper(freq='W'))['TOTAL_SALES'].sum().reset_index(), x="CALDAY", y="TOTAL_SALES")
fig.show()

#### Dla wszystkich sklepów

`testowanie`

In [12]:
liquidation_store_y = paragony.set_index('CALDAY').groupby(pd.Grouper(freq='W'))['TOTAL_SALES'].sum().reset_index().rename(columns = {'TOTAL_SALES':'VALUE'}).set_index('CALDAY')
liquidation_store_y.index.freq = 'W'
liquidation_store_X = paragony.set_index('CALDAY').groupby([pd.Grouper( freq='W')]).aggregate({'Wskaźnik_inflacji': 'mean', 'święto':(lambda x: sum(x)/paragony.STORE.nunique()), 'święta_szczególne':(lambda x: sum(x)/paragony.STORE.nunique())})

In [13]:
test_size=0.2
# PODZIAŁ NA ZBIÓR TRENINGOWY I TESTOWY I JEGO WIZUALIZACJA
y_train, y_test, X_train, X_test = temporal_train_test_split(liquidation_store_y,liquidation_store_X, test_size= test_size)
# Horyzont do predykcji
fh = ForecastingHorizon(X_test.dropna().index, is_relative=False)

`predykcja`

In [ ]:
forecaster_prophet = Prophet()
#forecaster_prophet = Prophet(add_country_holidays={'country_name': 'Poland'}, holidays_prior_scale=1)
forecaster_prophet.fit(y = y_train, X = X_train)

y_pred_prophet = forecaster_prophet.predict(X = X_test.dropna(),fh=fh) # predict


wykres(y_train,y_test,y_pred_prophet)

`prognoza w przód - luty-grudzień + analiza wizualna`

In [17]:
y_train, X_train = liquidation_store_y[liquidation_store_y.index < ''] , liquidation_store_X[liquidation_store_X.index < '']
X_train = X_train.fillna(method = 'bfill')
x = ...
X_train = X_train.fillna(x)    # Wypełnienie brakującej inflacji

# Tworzenie zbioru przyszłych X do predykcji
X_test = X.set_index('CALDAY')
X_test.index = pd.to_datetime(X_test.index)
X_test = X_test.groupby(pd.Grouper(freq='W')).aggregate({'Wskaźnik_inflacji':'mean', 'święto': (lambda x: sum(x)/X_test.STORE.nunique()),'święta_szczególne': (lambda x: sum(x)/X_test.STORE.nunique())})
X_test = X_test[-48:]

# Horyzont do predykcji
fh = ForecastingHorizon(y_train.index[-48:].shift(48), is_relative=False) # wcześniej 53
fh

In [ ]:
forecaster_prophet = Prophet(add_country_holidays={'country_name': 'Poland'})
forecaster_prophet.fit(y = y_train, X = X_train)

y_pred_prophet = forecaster_prophet.predict(X = X_test.dropna(),fh=fh) # predict

y_train = y_train.rename(columns={'TOTAL_SALES':'VALUE'})
y_test = y_test.rename(columns={'TOTAL_SALES':'VALUE'})
y_pred_prophet = y_pred_prophet.rename(columns={'TOTAL_SALES':'VALUE'})
liquidation_store_y = liquidation_store_y.rename(columns={'TOTAL_SALES':'VALUE'})

wykres(y_train,y_test,y_pred_prophet)

`Zapiasanie kroku`

In [28]:
all_stores = y_pred_prophet
all_stores['STORE_FORCOM_ID'] = 'all'

y_pred_intervals = forecaster_prophet.predict_interval(fh= fh, X=X_test.dropna())
y_pred_intervals.columns = ['lower','upper']
all_stores['lower'] = y_pred_intervals['lower']
all_stores['upper'] = y_pred_intervals['upper']


all_stores = all_stores.rename(columns = {'VALUE':'TOTAL_SALES'})

all_stores = convert_weekly_to_monthly(all_stores[['TOTAL_SALES', 'lower', 'upper']])
all_stores['STORE'] = 'all'

all_stores.head()

,TOTAL_SALES,STORE_FORCOM_ID,lower,upper
2023-02-05,4.692546e+07,all,4.189477e+07,5.233455e+07
2023-02-12,4.524156e+07,all,3.988520e+07,5.034114e+07
2023-02-19,4.527727e+07,all,4.001997e+07,5.040578e+07
2023-02-26,4.744995e+07,all,4.251718e+07,5.282548e+07
2023-03-05,5.026711e+07,all,4.476361e+07,5.548694e+07


In [34]:
y_train, X_train = liquidation_store_y[liquidation_store_y.index < ''] , liquidation_store_X[liquidation_store_X.index < '']
X_train = X_train.fillna(method = 'bfill')

# Tworzenie zbioru przyszłych X do predykcji
X_test = X.set_index('CALDAY')#.groupby(pd.Grouper(freq='W')).aggregate({'Wskaźnik_inflacji':'mean', 'święto':sum})
X_test.index = pd.to_datetime(X_test.index)
X_test = X_test.groupby(pd.Grouper(freq='W')).aggregate({'Wskaźnik_inflacji':'mean', 'święto': (lambda x: sum(x)/X_test.STORE.nunique()),'święta_szczególne': (lambda x: sum(x)/X_test.STORE.nunique())})
X_test = X_test[:6]

# Horyzont do predykcji
fh = ForecastingHorizon(y_train.index[-6:].shift(6), is_relative=False)

In [36]:
forecaster_prophet = Prophet(add_country_holidays={'country_name': 'Poland'})
forecaster_prophet.fit(y = y_train, X = X_train)

y_pred_prophet = forecaster_prophet.predict(X = X_test.dropna(),fh=fh)
y_pred_intervals = forecaster_prophet.predict_interval(fh= fh, X=X_test.dropna())
y_pred_intervals.columns = ['lower','upper']
y_pred_prophet['lower'] = y_pred_intervals['lower']
y_pred_prophet['upper'] = y_pred_intervals['upper']

all_stores_2 = y_pred_prophet
all_stores_2['STORE'] = 'all'
all_stores_2 = all_stores_2.rename(columns = {'VALUE':'TOTAL_SALES'})

all_stores_2 = convert_weekly_to_monthly(all_stores_2[['TOTAL_SALES', 'lower', 'upper']])
all_stores_2['STORE'] = 'all'
all_stores_2

14:01:56 - cmdstanpy - INFO - Chain [1] start processing
14:01:56 - cmdstanpy - INFO - Chain [1] done processing


`Połączenie prognoz styczniowej i luty-grudzień`

In [43]:
# PROGNOZA ZBIORCZA
all_stores = pd.concat([all_stores, all_stores_2]).sort_index()

`Sprawdzenie dopasowania na zbiorze testowym`

In [39]:
teeest = forecaster_prophet.predict(X = X_train,fh=X_train.index)
wykres(y_train, teeest)

#### Dla każdego ze sklepów

`Testowanie`

In [45]:
liquidation_store_y = paragony[paragony.STORE == sklep].set_index('CALDAY').groupby(pd.Grouper(freq='W'))['TOTAL_SALES'].sum().reset_index().set_index('CALDAY')
liquidation_store_y.index.freq = 'W'
liquidation_store_X =paragony[paragony.STORE == sklep].set_index('CALDAY').groupby([pd.Grouper( freq='W')]).aggregate({'Wskaźnik_inflacji':'mean', 'święto':sum, 'święta_szczególne':sum})

In [46]:
test_size=0.2
# PODZIAŁ NA ZBIÓR TRENINGOWY I TESTOWY I JEGO WIZUALIZACJA
y_train, y_test, X_train, X_test = temporal_train_test_split(liquidation_store_y,liquidation_store_X, test_size= test_size)
# Horyzont do predykcji
fh = ForecastingHorizon(X_test.dropna().index, is_relative=False)

In [ ]:
forecaster_prophet = Prophet(add_country_holidays={'country_name': 'Poland'})
forecaster_prophet.fit(y = y_train, X = X_train)

y_pred_prophet = forecaster_prophet.predict(X = X_test.dropna(),fh=fh) # predict

liquidation_store_y = liquidation_store_y.rename(columns = {'TOTAL_SALES':'VALUE'})
wykres(y_train.rename(columns = {'TOTAL_SALES':'VALUE'}),y_test.rename(columns = {'TOTAL_SALES':'VALUE'}),y_pred_prophet.rename(columns = {'TOTAL_SALES':'VALUE'}))

`Total sales`

`luty-grudzień`

In [ ]:
forecaster_prophet = Prophet(add_country_holidays={'country_name': 'Poland'})
przychody_sales = []
for sklep in X.STORE.unique():
    liquidation_store_y = paragony[paragony.STORE == sklep].set_index('CALDAY').groupby(pd.Grouper(freq='W'))['TOTAL_SALES'].sum().reset_index().set_index('CALDAY')
    liquidation_store_y.index.freq = 'W'
    liquidation_store_X =paragony[paragony.STORE == sklep].set_index('CALDAY').groupby([pd.Grouper( freq='W')]).aggregate({'Wskaźnik_inflacji':'mean', 'święto':sum, 'święta_szczególne':sum})
    
    # PODZIAŁ NA ZBIÓR TRENINGOWY I TESTOWY I JEGO WIZUALIZACJA
    y_train, X_train = liquidation_store_y[liquidation_store_y.index < ''] , liquidation_store_X[liquidation_store_X.index < '']
    X_train = X_train.fillna(method = 'bfill')
    X_train = X_train.fillna(xxx)                      # inflacja za brakujący okres
    # Horyzont do predykcji
    fh = ForecastingHorizon(y_train.index[-48:].shift(48), is_relative=False)
    
    # Tworzenie zbioru przyszłych X do predykcji
    X_test = X[X.STORE == sklep].set_index('CALDAY')
    X_test.index = pd.to_datetime(X_test.index)
    X_test = X_test.groupby(pd.Grouper(freq='W')).aggregate({'Wskaźnik_inflacji':'mean', 'święto':sum, 'święta_szczególne':sum})
    X_test = X_test
    
    # Predykcja przychodów
    forecaster_prophet.fit(y = y_train, X = X_train)
    y_pred_prophet = forecaster_prophet.predict(X = X_test.dropna(),fh=fh)
    y_pred_prophet['STORE'] = sklep
    y_pred_intervals = forecaster_prophet.predict_interval(fh= fh, X=X_test.dropna())
    y_pred_intervals.columns = ['lower','upper']
    y_pred_prophet['lower'] = y_pred_intervals['lower']
    y_pred_prophet['upper'] = y_pred_intervals['upper']
    
    przychody_sales.append(y_pred_prophet)
    print(sklep)
    
przychody_sales = pd.concat(przychody_sales)

# Zapisanie do pliku do generowania zbioru X
przychody_sales[['TOTAL_SALES','STORE']].reset_index().to_csv('przychody_sales.csv', index = False)

`styczeń`

In [ ]:
forecaster_prophet = Prophet(add_country_holidays={'country_name': 'Poland'})
przychody_sales_2 = []
for sklep in X.STORE.unique():
    liquidation_store_y = paragony[paragony.STORE == sklep].set_index('CALDAY').groupby(pd.Grouper(freq='W'))['TOTAL_SALES'].sum().reset_index().set_index('CALDAY')
    liquidation_store_y.index.freq = 'W'
    liquidation_store_X =paragony[paragony.STORE == sklep].set_index('CALDAY').groupby([pd.Grouper( freq='W')]).aggregate({'Wskaźnik_inflacji':'mean', 'święto':sum, 'święta_szczególne':sum})
    
    # PODZIAŁ NA ZBIÓR TRENINGOWY I TESTOWY I JEGO WIZUALIZACJA
    y_train, X_train = liquidation_store_y[liquidation_store_y.index < ''] , liquidation_store_X[liquidation_store_X.index < '']
    X_train = X_train.fillna(method = 'bfill')
    X_train = X_train.fillna(xxx)    # Inflacja za styczeń
    # Horyzont do predykcji
    fh = ForecastingHorizon(y_train.index[-6:].shift(6), is_relative=False)
    
    # Tworzenie zbioru przyszłych X do predykcji
    X_test = X[X.STORE == sklep].set_index('CALDAY')
    X_test.index = pd.to_datetime(X_test.index)
    X_test = X_test.groupby(pd.Grouper(freq='W')).aggregate({'Wskaźnik_inflacji':'mean', 'święto':sum, 'święta_szczególne':sum})
    X_test = X_test
    
    # Predykcja przychodów
    forecaster_prophet.fit(y = y_train, X = X_train)
    y_pred_prophet = forecaster_prophet.predict(X = X_test.dropna(),fh=fh)
    y_pred_prophet['STORE'] = sklep
    y_pred_intervals = forecaster_prophet.predict_interval(fh= fh, X=X_test.dropna())
    y_pred_intervals.columns = ['lower','upper']
    y_pred_prophet['lower'] = y_pred_intervals['lower']
    y_pred_prophet['upper'] = y_pred_intervals['upper']
    
    przychody_sales_2.append(y_pred_prophet)
    print(sklep)
    
przychody_sales_2 = pd.concat(przychody_sales_2)

`Grupowanie i połączenie prognoz styczniowej i luty-grudzień`

In [59]:
sal = []
for store in przychody_sales.STORE.unique():
    df = przychody_sales[przychody_sales.STORE_FORCOM_ID == store]
    df = convert_weekly_to_monthly(df[['TOTAL_SALES', 'lower', 'upper']])
    df['STORE'] = store
    sal.append(df)
przychody_sales = pd.concat(sal)

sal_2 = []
for store in przychody_sales_2.STORE.unique():
    df = przychody_sales_2[przychody_sales_2.STORE_FORCOM_ID == store]
    df = convert_weekly_to_monthly(df[['TOTAL_SALES', 'lower', 'upper']])
    df['STORE'] = store
    sal_2.append(df)
przychody_sales_2 = pd.concat(sal_2)

,TOTAL_SALES,lower,upper,STORE_FORCOM_ID
2023-02-28,384008.672381,339050.270067,430952.278142,1001
2023-03-31,476717.640189,428492.173213,525901.686469,1001
2023-04-30,408015.740580,358911.739056,457250.566089,1001
2023-05-31,382609.444580,332060.457804,432890.403496,1001
2023-06-30,379133.623709,330346.332903,427602.939928,1001
...,...,...,...,...
2023-01-31,488592.128877,405078.298841,574254.742647,4345
2023-01-31,672135.670249,581568.636872,764955.081440,4346
2023-01-31,816391.827216,647532.043863,983778.247770,4424
2023-01-31,233535.969564,175018.639939,292698.111936,4440


In [69]:
przychody = pd.concat([przychody_sales,przychody_sales_2]).sort_index().sort_values('STORE', kind='mergesort')

#### Złącznie wszystkich przychodów z przedziałem ufności do jednego pliku

In [82]:
przychody = pd.concat([all_stores,przychody])

In [83]:
przychody.reset_index().to_csv('predykcje_przychodow_all.csv', index = False)

`Total volume`

In [ ]:
forecaster_prophet = Prophet(add_country_holidays={'country_name': 'Poland'})
przychody_volume = []
for sklep in X.STORE.unique():
    liquidation_store_y = paragony[paragony.STORE == sklep].set_index('CALDAY').groupby(pd.Grouper(freq='W'))['TOTAL_VOLUME_SOLD'].sum().reset_index().set_index('CALDAY')
    liquidation_store_y.index.freq = 'W'
    liquidation_store_X =paragony[paragony.STORE == sklep].set_index('CALDAY').groupby([pd.Grouper( freq='W')]).aggregate({'Wskaźnik_inflacji':'mean', 'święto':sum})
    
    # PODZIAŁ NA ZBIÓR TRENINGOWY I TESTOWY I JEGO WIZUALIZACJA
    y_train, X_train = liquidation_store_y[liquidation_store_y.index < ''] , liquidation_store_X[liquidation_store_X.index < '']
    X_train = X_train.fillna(method = 'bfill')
    
    # Horyzont do predykcji
    fh = ForecastingHorizon(y_train.index[-53:].shift(53), is_relative=False)
    
    # Tworzenie zbioru przyszłych X do predykcji
    X_test = X[X.STORE_FORCOM_ID == sklep].set_index('CALDAY')
    X_test.index = pd.to_datetime(X_test.index)
    X_test = X_test.groupby(pd.Grouper(freq='W')).aggregate({'Wskaźnik_inflacji':'mean', 'święto':sum})
    X_test = X_test[-48:]
    
    # Predykcja przychodów
    forecaster_prophet.fit(y = y_train, X = X_train)
    y_pred_prophet = forecaster_prophet.predict(X = X_test.dropna(),fh=fh)
    y_pred_prophet['STORE'] = sklep
    
    przychody_volume.append(y_pred_prophet)
    print(sklep)
    
przychody_volume = pd.concat(przychody_volume)

In [46]:
przychody_volume.reset_index().to_csv('przychody_volume.csv', index = False)

### Łączenie danych w całość

`Wczytywanie dotychczasowego zbioru X`

In [62]:
X = pd.read_csv('Zbiór_X.csv')
X = X.drop_duplicates()
X = X.rename(columns = {'Wskaźnik cen konsumenta CPI (%, r/r)':'Wskaźnik_inflacji'})
X.CALDAY = pd.to_datetime(X.CALDAY)
X = X.set_index('CALDAY')

`Wczytywanie prognoz sprzedaży`

In [78]:
volume = pd.read_csv('przychody_volume.csv')
volume = volume.rename(columns = {'index':'CALDAY'})
volume.CALDAY = pd.to_datetime(volume.CALDAY)
volume = volume.set_index('CALDAY')
sales = pd.read_csv('przychody_sales.csv')
sales = sales.rename(columns = {'index':'CALDAY'})
sales.CALDAY = pd.to_datetime(sales.CALDAY)
sales = sales.set_index('CALDAY')

`Słownik do agregacji`

In [72]:
agg_dict = {...
}

`Obsłużenie braków danych i pustych pól`

In [73]:
for col in ['Wskaźnik_inflacji','TOTAL_SALES', 'TOTAL_VOLUME_SOLD']:
    try:
        X[col] = X[col].replace(0,np.nan).fillna(method='ffill').fillna(method='bfill') 
    except:
        pass

`Finalne połączenie i zapisanie do pliku`

In [76]:
zbiór_X = []
for sklep in X.STORE.unique():
    x = X[X.STORE == sklep].groupby(pd.Grouper(freq = 'W')).agg(agg_dict)
    do_poł = pd.merge(x, volume[volume.STORE == sklep]['TOTAL_VOLUME_SOLD'], left_index=True, right_index=True)
    do_poł = pd.merge(do_poł, sales[sales.STORE == sklep]['TOTAL_SALES'], left_index=True, right_index=True)
    zbiór_X.append(do_poł)
zbiór_X = pd.concat(zbiór_X)

In [79]:
zbiór_X.to_csv('./DANE_PRZYSZLE.csv')